<a href="https://colab.research.google.com/github/perfectism13/learning_colab/blob/master/C3D_rgbs_model/C3D_rgbs_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir(r'/content/drive/My Drive/state-farm-distracted-driver-detection/imgs/')
print(os.getcwd())
!ls
!pip install keras==2.1.5
import keras
print(keras.__version__)
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/state-farm-distracted-driver-detection/imgs
'(1-726)clean.mp4'		 my_model11.h5
'(1-726).mp4'			 my_model12.h5
'（726-800)clean.mp4'		 my_model13.h5
 （726-800）.mp4		 my_model1.h5
'(726-966)clean.mp4'		 my_model9.h5
'(726-966).mp4'			 sample_submission.csv
 driver_imgs_list_class_2.csv	 test
 driver_imgs_list_class.csv	 train
 driver_imgs_list_class.gsheet	 valid
 driver_imgs_list.csv		 vect
 driver_imgs_list_right.csv	 video_model_change_vd_data_aug.h5
 flow_xception_lock50.h5	 video_model_change_vd_data_aug_modified.h5
 flow_xception_lock70.h5	 video_model_change_vd.h5
 I3D_rgbs_model.h5		 video_model.h5
 my_model10.h5


Using TensorFlow backend.


2.1.5
Mon Feb 10 12:11:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
| 

In [2]:
!pip install sk-video

In [0]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
import keras

In [0]:
import numpy as np
np.random.seed(2017)

import os
import glob
import cv2
import math
import pickle
import pandas as pd
import random
from tqdm import tqdm
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, Adam, Nadam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

##构建模型

In [0]:
# -*- coding: utf-8 -*-
"""C3D model for Keras
# Reference:
- [Learning Spatiotemporal Features with 3D Convolutional Networks](https://arxiv.org/abs/1412.0767)
Based on code from @albertomontesg
"""

import skvideo.io
import keras.backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Dense, Dropout, Flatten
# from sports1M_utils import preprocess_input, decode_predictions
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D

WEIGHTS_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/sports1M_weights_tf.h5'

def C3D(weights='sports1M'):
    """Instantiates a C3D Kerasl model
    
    Keyword arguments:
    weights -- weights to load into model. (default is sports1M)
    
    Returns:
    A Keras model.
    
    """
    
    if weights not in {'sports1M', None}:
        raise ValueError('weights should be either be sports1M or None')
    
    if K.image_data_format() == 'channels_last':
        shape = (16,112,112,3)
    else:
        shape = (3,16,112,112)
        
    model = Sequential()
    model.add(Conv3D(64, 3, activation='relu', padding='same', name='conv1', input_shape=shape))
    model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same', name='pool1'))
    
    model.add(Conv3D(128, 3, activation='relu', padding='same', name='conv2'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool2'))
    
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3a'))
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3b'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool3'))
    
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4b'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool4'))
    
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=(0,1,1)))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool5'))
    
    model.add(Flatten())
    
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(487, activation='softmax', name='fc8'))

    if weights == 'sports1M':
        weights_path = get_file('sports1M_weights_tf.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models',
                                md5_hash='b7a93b2f9156ccbebe3ca24b41fc5402')
        
        model.load_weights(weights_path)
    
    return model
    
# if __name__ == '__main__':
#     model = C3D(weights='sports1M')

##构建模型，设定可训练的层数

In [6]:
from keras import backend as K
K.clear_session()

In [7]:
from keras.models import load_model
import h5py
import os
from keras.layers import regularizers, GlobalAveragePooling2D, Flatten,Input
from keras.models import Model
# x = Input((16,112,112,3))
base_model = C3D(weights='sports1M')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc6').output)
x = Dropout(0.5)(model.output)
# x = Dropout(0.5)(x)
x = Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(x)
# x = Flatten()(x)
model = Model(base_model.input,x)
for i in range(6):
     model.layers[i].trainable = False
model.summary()
len(model.layers)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_input (InputLayer)     (None, 16, 112, 112, 3)   0         
_________________________________________________________________
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
___________________

19

#切分数据

In [8]:
# divide drivers
# unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
# unique_list_valid = ['p016', 'p021', 'p022', 'p024']
unique_list_train = ['p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
unique_list_valid = ['p002', 'p012', 'p014', 'p015']
#print (unique_list_train, unique_list_valid)

# get index: driver_id, class, image name
index = os.path.join('.', 'driver_imgs_list_class.csv')

# build the driver id dictionary and class dictionary
f = open(index, 'r')
id_dict = dict()
class_dict = dict()
lines = f.readlines()
# for line in lines[1:]:
#     arr = line.strip().split(',')
#     if arr[0] not in id_dict.keys():
#         id_dict[arr[0]] = [line]
#     else:
#         id_dict[arr[0]].append(line)
#     if arr[1] not in class_dict.keys():
#         class_dict[arr[1]] = [line]
#     else:
#         class_dict[arr[1]].append(line)
# f.close()

# split the train list and valid list by id
train_list = []
valid_list = []
# for id in id_dict.keys():
#     if id in unique_list_train:
#         train_list.extend(id_dict[id])
#     elif id in unique_list_valid:
#         valid_list.extend(id_dict[id])

# for cla in class_dict.keys():
#   for id in id_dict.keys():
#     if id in unique_list_train:
#         train_list.extend(class_dict[cla])
#     elif id in unique_list_valid:
#         valid_list.extend(class_dict[cla])
# for line in lines[1:]:
#     arr = line.strip().split(',')
#     if arr[0]  in unique_list_train:
#         train_list.append(line)
#     elif arr[0]  in unique_list_valid:
#       valid_list.append(line)

for i in range(22426):
  if(i+17>=22425):
    break
  arr = lines[i+1].strip().split(',')
  for j in range(16) :
    flags = 0
    arr1 = lines[i+1+j].strip().split(',')
    arr2 = lines[i+2+j].strip().split(',')
    if(arr1[1]==arr2[1]):
        flags = 1
    elif(arr1[1]!=arr2[1]):
        flags = 0
  if(flags == 1):
    meta = []
    for k in range(16):
        meta.append(lines[i+1+k].strip().split(','))
    if arr[0] in unique_list_train:
      train_list.append(meta)
    elif arr[0] in unique_list_valid:
      valid_list.append(meta)
f.close()

random.shuffle(train_list)
random.shuffle(valid_list)
print (len(train_list), len(valid_list))
print(len(train_list[1]))

18302 3299
16


# 转换为One Hot Encode标签

对分类标签进行One Hot Encode的函数如下。

In [9]:
# one hot encode the class label

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])   
def one_hot_encode(x):
    return lb.transform(x)
t = one_hot_encode(['c1', 'c2'])
print(t)

[[0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]]


#获取rgbs

In [0]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
from keras.preprocessing import image
from random import choice
# global num_frames
# num_frames = 16
# image rotation
def rotate(frame, num_frames,degree, row_axis=0, col_axis=1, channel_axis=2, fill_mode='wrap', cval=0.):
    theta = np.pi / 180 * degree
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]])
    h, w = frame[0].shape[row_axis], frame[0].shape[col_axis]
    transform_matrix = image.transform_matrix_offset_center(rotation_matrix, h, w)
    for i in range(num_frames):
      frame[i] = image.apply_transform(frame[i], transform_matrix, channel_axis, fill_mode, cval)
    return frame

# image shift
def shift(frame,num_frames, wshift, hshift, row_axis=0, col_axis=1, channel_axis=2, fill_mode='wrap', cval=0.):
    h, w = frame[0].shape[row_axis], frame[0].shape[col_axis] #读取图片的高和宽
    tx = hshift * h #高偏移大小，若不偏移可设为0，若向上偏移设为正数
    ty = wshift * w #宽偏移大小，若不偏移可设为0，若向左偏移设为正数
    translation_matrix = np.array([[1, 0, tx],
                                  [0, 1, ty],
                                  [0, 0, 1]])
    transform_matrix = translation_matrix  
    for i in range(num_frames):
      frame[i] = image.apply_transform(frame[i], transform_matrix, channel_axis, fill_mode, cval)
    return frame

#左右或者上下翻转
def flip(frame,num_frames,tar):
  for i in range(num_frames):
    frame[i] = cv2.flip(frame[i],tar)
  return frame
#水平翻转90°
def transpose(frame,num_frames,tar):
  if(tar==1):
    for i in range(num_frames):
      frame[i] = cv2.transpose(frame[i])
  return frame

def get_rgbs_aug(path):
  num_frames = 16
  frame = []
  for i in range(num_frames):
    image = cv2.imread(path[i])
    image = np.array(image, dtype=np.float32)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = cv2.resize(image, (224, 224))
    frame.append(image)
  #random rotate
  deg = random.uniform(-10, 10) #random rotate limit
  frame = rotate(frame,num_frames, deg)
  #random shift
  wshift = random.uniform(-0.1, 0.1)
  hshift = random.uniform(-0.1, 0.1)
  frame = shift(frame,num_frames, wshift, hshift)
  #random transpose
  foo = [0,1]
  tar = choice(foo)
  frame = transpose(frame,num_frames,tar)
  #random flip
  foo = [0,-1,1]
  tar = choice(foo)
  frame = flip(frame,num_frames,tar)
  # reduce size
  for i in range(num_frames):
    frame[i] = cv2.resize(frame[i], (112, 112))
  # normalization
  frame = np.array(frame, dtype=np.float32)  
  frame /= 127.5
  frame -= 1.
  return frame

def get_rgbs(path):
  num_frames = 16
  frame = []
  for i in range(num_frames):
    image = cv2.imread(path[i])
    image = cv2.resize(image, (112, 112))
    image = np.array(image, dtype=np.float32)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    frame.append(image)
  # normalization
  frame = np.array(frame, dtype=np.float32)  
  frame /= 127.5
  frame -= 1.
  return frame


In [11]:
foo = [0,1,-1]
from random import choice
print (choice(foo))

-1


#训练数据生成器

In [0]:
# define my train data generator
# global num_frames
# num_frames = 16
def train_gen(batch_size):
    #random.shuffle(train_list) # 每一代都随机shuffle训练集
    current = 0
    num_frames = 16
    while 1:
        x = []
        y = []
        path = []
        while len(y) < batch_size:
            for i in range(num_frames):
              # line=train_list[current+i]
              # arr=line.strip().split(',')
              # mpath=os.path.join('.', 'train', str(arr[1]), str(arr[2]))
              # path.append(mpath)
              line=train_list[current][i]
              # arr=line.strip().split(',')
              mpath=os.path.join('.', 'train', str(line[1]), str(line[2]))
              path.append(mpath)
            rgbs = get_rgbs_aug(path)
            # if random.random()>0.5:
            #     line2 = random.choice(class_dict[arr[1]])
            #     bname = line2.strip().split(',')[2]
            #     path2 = os.path.join('.', 'train', str(arr[1]), str(bname))
            #     img2 = get_im_cv2_aug(path2, 299)
            #     left = img[:, :150, :]
            #     right = img2[:, 150:, :]
            #     img = np.concatenate((left, right), axis=1)
            x.append(rgbs)
            label = one_hot_encode([str(line[1])])[0]
            y.append(label)
            current += 1
            if current >= len(train_list):
                current = 0
        x = np.array(x)
        # print(x.shape,y.shape)
        img_size = 112
        # num_frames = 16
        x = x.reshape(batch_size,num_frames, img_size, img_size, 3) 
        y = np.array(y, dtype = np.uint8)
        y = y.reshape(batch_size, 10)
        yield (x, y)

#验证数据生成器

In [0]:
# define my validation data generator
# global num_frames
# num_frames = 16
def valid_gen(batch_size):
    #random.shuffle(train_list) # 每一代都随机shuffle训练集
    current = 0
    num_frames = 16
    while 1:
        x = []
        y = []
        path = []
        while len(y) < batch_size:
            for i in range(num_frames):
              # line=valid_list[current+i]
              # arr=line.strip().split(',')
              # mpath=os.path.join('.', 'train', str(arr[1]), str(arr[2]))
              # path.append(mpath)
              line=valid_list[current][i]
              # arr=line.strip().split(',')
              mpath=os.path.join('.', 'train', str(line[1]), str(line[2]))
              path.append(mpath)
            rgbs = get_rgbs(path)
            x.append(rgbs)
            label = one_hot_encode([str(line[1])])[0]
            y.append(label)
            current += 1
            if current >= len(valid_list):
                current = 0
        x = np.array(x)
        # print(x.shape,y.shape)
        img_size = 112
        # num_frames = 16
        x = x.reshape(batch_size,num_frames, img_size, img_size, 3) 
        y = np.array(y, dtype = np.uint8)
        y = y.reshape(batch_size, 10)
        yield (x, y)

#验证数据类型

In [14]:
current = 0
path = []
num_frames=16
for i in range(num_frames):
  line=train_list[current][i]
  print(line)
  # arr=line.strip().split(',')
  mpath=os.path.join('.', 'train', str(line[1]), str(line[2]))
  path.append(mpath)
for i in range(num_frames):
  line=valid_list[current][i]
  print(line)
  # arr=line.strip().split(',')
  mpath=os.path.join('.', 'train', str(line[1]), str(line[2]))
  path.append(mpath)

rgbs_aug = get_rgbs_aug(path)
rgbs = get_rgbs(path)
print(rgbs_aug.shape)
print(rgbs.shape)

['p035', 'c4', 'img_62615.jpg']
['p035', 'c4', 'img_22165.jpg']
['p035', 'c4', 'img_32465.jpg']
['p035', 'c4', 'img_101957.jpg']
['p035', 'c4', 'img_30451.jpg']
['p035', 'c4', 'img_9086.jpg']
['p035', 'c4', 'img_28423.jpg']
['p035', 'c4', 'img_31570.jpg']
['p035', 'c4', 'img_12719.jpg']
['p035', 'c4', 'img_72376.jpg']
['p035', 'c4', 'img_64670.jpg']
['p035', 'c4', 'img_84456.jpg']
['p035', 'c4', 'img_8574.jpg']
['p035', 'c4', 'img_39421.jpg']
['p035', 'c4', 'img_8119.jpg']
['p035', 'c4', 'img_52001.jpg']
['p014', 'c4', 'img_71169.jpg']
['p014', 'c4', 'img_82074.jpg']
['p015', 'c4', 'img_73258.jpg']
['p015', 'c4', 'img_50429.jpg']
['p015', 'c4', 'img_51146.jpg']
['p015', 'c4', 'img_24154.jpg']
['p015', 'c4', 'img_31684.jpg']
['p015', 'c4', 'img_100778.jpg']
['p015', 'c4', 'img_2071.jpg']
['p015', 'c4', 'img_36357.jpg']
['p015', 'c4', 'img_12333.jpg']
['p015', 'c4', 'img_41379.jpg']
['p015', 'c4', 'img_12476.jpg']
['p015', 'c4', 'img_58436.jpg']
['p015', 'c4', 'img_53991.jpg']
['p015', '

#训练

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
model.compile(optimizer=Nadam(),loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# global num_frames
# num_frames =16
def learning_rate(epoch):
    ini_lr = 0.002
    lr = ini_lr * pow(10, -epoch)  
    return lr

cp = ModelCheckpoint(filepath="C3D_rgbs_model.h5", save_best_only=True)
es = EarlyStopping()
lrs = LearningRateScheduler(learning_rate)
hist = model.fit_generator(train_gen(32), steps_per_epoch = 572, epochs = 5, workers=4, max_q_size=20, use_multiprocessing=True, validation_data = valid_gen(32), validation_steps = 104, callbacks=[cp, es, lrs])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=572, epochs=5, workers=4, use_multiprocessing=True, validation_data=<generator..., validation_steps=104, callbacks=[<keras.ca..., max_queue_size=20)`
  del sys.path[0]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
 28/572 [>.............................] - ETA: 9:13:13 - loss: 14.1362 - categorical_accuracy: 0.1060

#模型结果

In [0]:
from matplotlib import pyplot as plt
print (hist.history)
plt.figure(1)
plt.plot (hist.history['loss'],label='loss')
plt.plot (hist.history['val_loss'], label='val_loss')
plt.legend(bbox_to_anchor=(1,1),#图例边界框起始位置
                 loc="upper right",#图例的位置
                 ncol=1,#列数
                 mode="None",#当值设置为“expend”时，图例会水平扩展至整个坐标轴区域
                 borderaxespad=0,#坐标轴和图例边界之间的间距
                 shadow=False,#是否为线框添加阴影
                 fancybox=True)#线框圆角处理参数
plt.show()



In [0]:
plt.figure(2)
plt.plot (hist.history['categorical_accuracy'], label='categorical_accuracy')
plt.plot (hist.history['val_categorical_accuracy'], label='val_categorical_accuracy')
plt.legend(bbox_to_anchor=(1,0),#图例边界框起始位置
                 loc="lower right",#图例的位置
                 ncol=1,#列数
                 mode="expend",#当值设置为“expend”时，图例会水平扩展至整个坐标轴区域
                 borderaxespad=0,#坐标轴和图例边界之间的间距
                 shadow=False,#是否为线框添加阴影
                 fancybox=True)#线框圆角处理参数
plt.show()